# 51. Introduction to Cumulative Aggregations and Ranking in SQL Queries

# 52. Overview of CTAS to create tables based on Query Results

In [ ]:
%%sql
CREATE TABLE order_count_by_status
AS
SELECT orders.order_status, count(*) AS order_count
FROM orders
GROUP BY 1;

SELECT * FROM order_count_by_status;

CREATE TABLE orders_stg
AS
SELECT * FROM orders WHERE 1=2;

-- SELECT o.order_date,
--     round(sum(oi.order_item_subtotal)::numeric, 2) AS order_revenue
-- FROM orders AS o
--     JOIN order_items AS oi
--         ON o.order_id = oi.order_item_order_id
-- WHERE o.order_status IN ('COMPLETE', 'CLOSED')
-- GROUP BY 1

In [ ]:
%%sql
SELECT * FROM orders_stg;

In [ ]:
%%sql
SELECT * FROM order_count_by_status;

# 53. Create Tables for Cumulative Aggregations and Ranking

In [ ]:
%%sql
CREATE TABLE daily_revenue
AS
SELECT o.order_date,
       ROUND(SUM(oi.order_item_subtotal)::NUMERIC, 2) AS order_revenue
FROM orders AS o
JOIN order_items AS oi
    ON o.order_id = oi.order_item_order_id
WHERE o.order_status IN ('COMPLETE', 'CLOSED')
GROUP BY o.order_date;

In [ ]:
%%sql
SELECT * FROM daily_revenue
ORDER BY order_date;

In [ ]:
%%sql
CREATE TABLE daily_product_revenue
AS
SELECT o.order_date,
    oi.order_item_product_id,
    round(sum(oi.order_item_subtotal)::numeric, 2) AS order_revenue
FROM orders AS o
    JOIN order_items AS oi
        ON o.order_id = oi.order_item_order_id
WHERE o.order_status IN ('COMPLETE', 'CLOSED')
GROUP BY 1, 2;

In [ ]:
%%sql
SELECT * FROM daily_product_revenue
ORDER BY 1, 3 DESC;

# 54. Overview of OVER and PARTITION BY Clause in SQL Queries

In [ ]:
%%sql
-- SELECT * FROM daily_revenue
-- ORDER BY 1;

SELECT to_char(dr.order_date::timestamp, 'yyyy-MM') AS order_month,
    sum(order_revenue) AS order_revenue
FROM daily_revenue AS dr
GROUP BY 1
ORDER BY 1;

In [ ]:
%%sql
SELECT to_char(dr.order_date::timestamp, 'yyyy-MM') AS order_month,
    dr.order_date,
    dr.order_revenue,
    sum(order_revenue) OVER (
        PARTITION BY to_char(dr.order_date::timestamp, 'yyyy-MM')) AS monthly_order_revenue
FROM daily_revenue AS dr
ORDER BY 1;

# 55. Compute Total Aggregation using OVER and PARTITION BY in SQL Queries

In [ ]:
%%sql
SELECT dr.*,
       sum(order_revenue) OVER (PARTITION BY 1) as total_revenue
FROM daily_revenue AS dr
ORDER BY 1;

# 56. Overview of Ranking in SQL
# 57. Compute Global Ranks using SQL

In [ ]:
%%sql
-- SELECT count(*) FROM daily_product_revenue;

SELECT * FROM daily_product_revenue
ORDER BY order_date, order_revenue DESC;

In [ ]:
%%sql
SELECT order_date,
        order_item_product_id,
        order_revenue,
        rank() OVER (ORDER BY order_revenue) AS rnk,
        dense_rank() OVER (ORDER BY order_revenue DESC) AS drnk
FROM daily_product_revenue
WHERE order_date = '2014-01-01 00:00:00.0'
ORDER BY order_revenue DESC;

# 58. Compute Ranks based on key using SQL

In [ ]:
%%sql
SELECT order_date,
        order_item_product_id,
        order_revenue,
        rank() OVER (
            PARTITION BY order_date
            ORDER BY order_revenue DESC) AS rnk,
        dense_rank() OVER (
            PARTITION BY order_revenue
            ORDER BY order_revenue DESC) AS drnk
FROM daily_product_revenue
WHERE to_char(order_date, 'yyyy-MM') = '2014-01'
ORDER BY order_revenue DESC;

# 59. Rules and Restrictions to Filter Data based on Ranks in SQL
# 60. Filtering based on Global Ranks using Nested Queries and CTEs in SQL

In [ ]:
%%sql
SELECT * FROM (
SELECT order_date,
        order_item_product_id,
        order_revenue,
        rank() OVER (ORDER BY order_revenue) AS rnk,
        dense_rank() OVER (ORDER BY order_revenue DESC) AS drnk
FROM daily_product_revenue
WHERE order_date = '2014-01-01 00:00:00.0'
ORDER BY order_revenue DESC
) AS q
WHERE drnk <= 5
ORDER BY order_revenue DESC
;

In [ ]:
%%sql
WITH daily_product_revenue_ranks AS(
    SELECT order_date,
        order_item_product_id,
        order_revenue,
        rank() OVER (ORDER BY order_revenue DESC) AS rnk,
        dense_rank() OVER (ORDER BY order_revenue DESC) AS drnk
FROM daily_product_revenue
WHERE order_date = '2014-01-01 00:00:00.0'
ORDER BY order_revenue DESC
) SELECT * FROM daily_product_revenue_ranks
WHERE drnk <= 5
ORDER BY order_revenue DESC;

# 61. Filtering based on Ranks per Partition using Nested Queries and CTEs in SQL

In [ ]:
%%sql
SELECT * FROM (
SELECT order_date,
    order_item_product_id,
    order_revenue,
    rank() OVER (
        PARTITION BY order_date
        ORDER BY order_revenue DESC
    ) AS rnk,
    dense_rank() OVER (
        PARTITION BY order_date
        ORDER BY order_revenue DESC
    ) AS drnk
FROM daily_product_revenue
WHERE to_char(order_date::date, 'yyyy-MM') = '2014-01'
) AS q
WHERE drnk <= 5
ORDER BY order_date, order_revenue DESC
;

In [ ]:
%%sql
WITH daily_product_revenue_ranks AS (
    SELECT order_date,
    order_item_product_id,
    order_revenue,
    rank() OVER (
        PARTITION BY order_date
        ORDER BY order_revenue DESC
    ) AS rnk,
    dense_rank() OVER (
        PARTITION BY order_date
        ORDER BY order_revenue DESC
    ) AS drnk
FROM daily_product_revenue
WHERE to_char(order_date::date, 'yyyy-MM') = '2014-01'
) SELECT * FROM daily_product_revenue_ranks
WHERE drnk <= 5
ORDER BY order_date, order_revenue DESC;

# 62. Create Students table with Data for ranking using SQL

In [ ]:
%%sql
CREATE TABLE student_scores (
    student_id INT PRIMARY KEY,
    student_score INT
);

INSERT INTO student_scores VALUES
(1, 980),
(2, 960),
(3, 960),
(4, 990),
(5, 920),
(6, 960),
(7, 980),
(8, 960),
(9, 940),
(10, 940);

In [ ]:
%%sql
SELECT * FROM studet_scores
ORDER BY student_score DESC;

# 63. Difference between rank and dense rank using SQL

In [2]:
%%sql
SELECT student_id,
    student_score,
    rank() OVER (ORDER BY student_score DESC) AS student_rank,
    dense_rank() OVER (ORDER BY student_score DESC) AS student_drank
FROM student_scores
ORDER BY student_score DESC;

,student_id,student_score,student_rank,student_drank
0,4,990,1,1
1,7,980,2,2
2,1,980,2,2
3,6,960,4,3
4,2,960,4,3
5,3,960,4,3
6,8,960,4,3
7,10,940,8,4
8,9,940,8,4
9,5,920,10,5
